Выберем алгоритм SGNS, поскольку из результатов статьи следует, что именно этот алгоритм лучше всего выявляет семантические сдвиги

In [252]:
!pip install matplotlib

  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/da/36/236fbd868b6c91309a5206bd90c3f881f4f44b2d997cd1d6239ef652f878/matplotlib-3.9.4-cp39-cp39-macosx_11_0_arm64.whl.metadata
  Using cached matplotlib-3.9.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (11 kB)
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/ec/22/19f5b948367ab5260fb41d842c7a78dae645603881ea6bc39738bcfcabf6/contourpy-1.3.0-cp39-cp39-macosx_11_0_arm64.whl.metadata
  Using cached contourpy-1.3.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (5.4 kB)
  Obtaining dependency information for cycler>=0.10 from https://files.pythonhosted.org/packages/e7/05/c19819d5e3d95294a6f5947fb9b9629efb316b96de511b418c53d245aae6/cycler-0.12.1-py3-none-any.whl.metadata
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/d2/c7/3bddafbb95447f6fb

In [2]:
from gensim.models.word2vec import LineSentence
!pip install gensim



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [12]:
!pip install cython


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import json
from gensim.test.utils import datapath
from gensim import utils
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import numpy as np
from scipy.linalg import orthogonal_procrustes
from scipy.spatial.distance import cosine
import os


/Users/edemmutalup/Desktop/semantic_changes/scripts/model_training/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
digits = {1: "one", 2: "two", 3: "three"}
time = 2

In [3]:
data = {"fiction_and_blogs": 0, "fiction_and_news": 0}

for type in data:
    tmp_data = []
    for i in range(time):
        json_file_path = f"../../processed_data_upgraded/training_data/{type}_datum_{digits[i + 1]}.json"
        with open(json_file_path, 'r', encoding='utf-8') as f:
            datum = json.load(f)
        tmp_data.append(datum)
    data[type] = tmp_data

In [4]:
def quality_research(data, ep_diap):

    qual = {"fiction_and_blogs": [], "fiction_and_news": []}

    for type in qual:

        for i in range(time):
            scores = [[], [], []]
            for ep in range(*ep_diap):
                model = Word2Vec(
                sentences=data[type][i],
                vector_size=300,
                window=4,
                min_count=3,
                sg=1,
                workers=6,
                epochs=ep
                )
                similarity_score = model.wv.evaluate_word_pairs("../../raw_data/wordsim353-russian.txt")
                pearson_result, spearman_result = similarity_score[0][0], similarity_score[1][0]
                scores[0].append(ep)
                scores[1].append(pearson_result)
                scores[2].append(spearman_result)

            qual[type].append(scores)

    return qual

In [13]:
ep_range = (10, 200, 5)

quality = quality_research(data, ep_range)

In [17]:
len(quality["fiction_and_news"][0][0])

38

In [18]:
min_quot = {"fiction_and_blogs": [], "fiction_and_news": []}

for type in min_quot:
    for idx in range(time):
        size = len(quality[type][idx][0])
        tmp = []
        for i in range(size):
            val = min(quality[type][idx][1][i], quality[type][idx][2][i])
            tmp.append(val)
        min_quot[type].append(tmp)


In [23]:
avg_quot = {"fiction_and_blogs": [], "fiction_and_news": []}

for type in avg_quot:
    for idx in range(time):
        size = len(quality[type][idx][0])
        tmp = []
        for i in range(size):
            val = (quality[type][idx][1][i] + quality[type][idx][2][i]) / 2.0
            tmp.append(val)
        avg_quot[type].append(tmp)


In [24]:
import matplotlib.pyplot as plt

def plot_graph(x_values, y_values, title="График", xlabel="X", ylabel="Y", output_path="pic"):

    plt.figure(figsize=(10, 6), dpi=300)

    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    plt.plot(x_values, y_values, marker='o', linestyle='-', color='blue')
    plt.grid(True)
    plt.tight_layout()

    full_output_path = "../../processed_data_upgraded/pics/Качество корпуса от количества эпох" + output_path + ".png"

    output_dir = os.path.dirname(full_output_path)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    plt.savefig(full_output_path, bbox_inches='tight')
    plt.close()


In [25]:
dict_for_title = {"fiction_and_blogs": "Худож. литература + Блоги", "fiction_and_news": "Худож. литература + ЧасКор (новости)"}
dict_for_time = {0: "до 2000", 1: "с 2000 по 2015"}

In [26]:
for type in min_quot:
    for idx in range(time):
        plot_graph(quality[type][idx][0],
                   avg_quot[type][idx],
                   title=dict_for_title[type] + " " + dict_for_time[idx],
                   xlabel="Эпохи",
                   ylabel="Среднее коэффициентов Пирсона и Спирмена",
                   output_path=dict_for_title[type] + " " + dict_for_time[idx] + " среднее")


In [22]:
with open(f"../../processed_data_upgraded/quality_of_training.json", 'w', encoding='utf-8') as file:
        json.dump(quality, file, ensure_ascii=False, indent=2)

На основе графиков, можно сделать вывод, что самое удачное количество эпох для Худож. литература + ЧасКор (новости) в период до 2000 - 185, в период с 2000 по 2015 - 140

In [37]:
model_until2000 = Word2Vec(
            sentences=data["fiction_and_news"][0],
            vector_size=300,
            window=4,
            min_count=3,
            sg=1,
            workers=6,
            epochs=185
            )

In [39]:
model_after2000 = Word2Vec(
            sentences=data["fiction_and_news"][1],
            vector_size=300,
            window=4,
            min_count=3,
            sg=1,
            workers=6,
            epochs=140
            )

In [43]:
model_until2000.save(f"../../processed_data_upgraded/models/model_until2000.model")
model_after2000.save(f"../../processed_data_upgraded/models/model_after2000.model")

In [40]:
similarity_score = model_after2000.wv.evaluate_word_pairs("../../raw_data/wordsim353-russian.txt")
pearson_result, spearman_result = similarity_score[0][0], similarity_score[1][0]
print(pearson_result, spearman_result)

0.3409034748075943 0.3497793162105742


In [46]:
def align_embeddings(model1, model2, common_vocab):
    """
    Выравнивает эмбеддинги модели model1 относительно модели model2 на основе общего словаря.

    Аргументы:
      model1: gensim-модель (или её KeyedVectors), которую нужно преобразовать.
      model2: gensim-модель (или её KeyedVectors), относительно которой производится выравнивание.
      common_vocab: список слов, присутствующих в обеих моделях.

    Возвращает:
      R: матрица преобразования, которую можно применить ко всем векторным представлениям model1.
    """
    # Формируем матрицы векторов из общего словаря
    X = np.array([model1.wv[word] for word in common_vocab])
    Y = np.array([model2.wv[word] for word in common_vocab])

    # Вычисляем матрицу R, решающую задачу Orthogonal Procrustes
    R, _ = orthogonal_procrustes(X, Y)
    return R

In [47]:
def transform_model(model, R):
    """
    Применяет матрицу преобразования R ко всем эмбеддингам модели (model.wv).
    Обратите внимание: этот метод изменяет модель на месте.
    """
    # Для gensim 4.x используем model.wv.index_to_key для итерации по словам
    for word in model.wv.index_to_key:
        model.wv[word] = np.dot(model.wv[word], R)

In [48]:
common_vocab = [word for word in model_until2000.wv.index_to_key if word in model_after2000.wv]

R = align_embeddings(model_until2000, model_after2000, common_vocab)
R = align_embeddings(model_until2000, model_after2000, common_vocab)

transform_model(model_until2000, R)

In [49]:
def compute_semantic_shift(model1, model2, common_vocab):
    """
    Для каждого слова из общего словаря вычисляет косинусное сходство между
    его вектором в model1 и вектором в model2.

    :param model1: Выровненная gensim-модель для первого периода.
    :param model2: Выровненная gensim-модель для второго периода.
    :param common_vocab: Список слов, присутствующих в обеих моделях.
    :return: Список кортежей (слово, косинусное сходство), отсортированный по возрастанию сходства.
    """
    shifts = {}
    for word in common_vocab:
        # Получаем векторы для слова из обеих моделей
        vec1 = model1.wv[word]
        vec2 = model2.wv[word]
        # Вычисляем косинусное сходство:
        # Обратите внимание: функция cosine из scipy.spatial.distance возвращает расстояние,
        # т.е. 0 - полностью совпадают, а 1 - полностью противоположны.
        # Поэтому, чтобы получить сходство, можно использовать: sim = 1 - cosine(vec1, vec2)
        sim = 1 - cosine(vec1, vec2)
        shifts[word] = sim

    # Сортируем слова по сходству (минимальное сходство означает наибольший сдвиг)
    sorted_shifts = sorted(shifts.items(), key=lambda item: item[1])
    return sorted_shifts

In [68]:
shifted_words = compute_semantic_shift(model_until2000, model_after2000, common_vocab)
print("Наиболее изменившиеся слова:")
for word, sim in shifted_words[:10]:
    if sim < 0.1: continue
    print(f"{word}: сходство {sim:.4f}")

2019
Наиболее изменившиеся слова:
живопись: сходство 0.1549
облегчение: сходство 0.2030
меняться: сходство 0.2086
подозревать: сходство 0.2095
пожаловать: сходство 0.2147
безумный: сходство 0.2229
удалить: сходство 0.2264
многое: сходство 0.2281
вывозить: сходство 0.2299
гений: сходство 0.2329


In [51]:
print(f"Общий словарь содержит {len(common_vocab)} слов.")

Общий словарь содержит 2019 слов.


In [65]:
similar_words = (model_until2000.wv.most_similar("пол", topn=10))
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

опершись: 0.4443
торопливо: 0.4439
обит: 0.4309
потолок: 0.4169
улечься: 0.4149
взад: 0.4133
штора: 0.4071
наоборот: 0.4032
готовить: 0.4018
абажур: 0.3967


In [66]:
similar_words = (model_after2000.wv.most_similar("пол", topn=10))
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

рон: 0.5251
представительница: 0.4393
ладно: 0.3914
полезно: 0.3743
саймон: 0.3645
футболист: 0.3504
слабый: 0.3484
зураб: 0.3427
живописный: 0.3387
разбросан: 0.3379
